<a href="https://colab.research.google.com/github/SophieSchau/MRF_demo_ISMRM2022/blob/main/MRF_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MRF processing demo
This notebook will demonstrate the processing pipeline used in the abstract *Toward a 1-minute high-resolution brain exam - MR Fingerprinting with ML-synthesized contrasts and fast reconstruction* presented at the 2022 ISMRM Annual meeting in London (program number 53).




## Overview
This project is aimed at translating highly undrsampled MRF to a clinically feasible tool, by building a robust reconstruction pipeline that is portable between different compute systems (research lab, hospital, high performance computing cluster, collaborators, etc...). To achieve this these core objectives were set:

- The pipeline should run smoothly on multiple systems.
- The pipeline should be easy to upgrade when the sequence, the reconstruction method, or the synthesis method is changed.
- The pipeline should be able to provide an image to send back to the scanner within 5 min.
- The pipeline should be able to send a series of images to PACS within ~30min.
- The pipeline should run on hardware available in clinical settings (for now, this means an 11GB GPU).

This modular MRF processing pipeline includes 4 steps:

1.   Download demo data and code 
2.   Reconstruct and get coil compression from calibration scan
3.   Reconstruct MRF (fast subspace basis reconstruction)
4.   Synthesize clinical contrasts

Each step will be demonstrated here, and documentation on how to run the equivalent Docker/Singularity containers on your own machine will be explained in a final section of this document.


## Data and code download 
*Run the next code block before reading this! The data takes a couple of minutes to download*

For this demo we have prepared a dataset consisting of five files:
1. `mrf_raw.npy` <- contains MRF kspace data
2. `calib_raw.npy` <- contains calibration GRE kspace data
3. `calib_nse.npy` <- contains noise measurement from the calibration GRE
4. `traj_grp16_inacc2.mat` <- contains the trajectory coordinates for the MRF
5. `subspace_basis_grp16.mat` <- contains the temporal basis components for the MRF


`mrf_raw.npy`, `calib_raw.npy`, and `calib_nse.npy` have been generated from ScanArchive files, which is the raw data output format used for MRI scanners by GE Healthcare. The conversion was done using containerized code provided by GE Healthcare. Since containers cannot easily be executed in Google Colab we skipped this step in the demo. However, if you want to run the conversion on your own system, you can run the containerized version on your own computer as explained in the final section of this demo.

*If your data hasn't downloaded yet, please sing happy birthday to yourself to pass the time, your birthday is soon/was recently anyway!*

In [ ]:
%%bash

mkdir tmp
cd tmp
wget -q -O data.zip -L https://stanfordmedicine.box.com/shared/static/61tx7yzaxrcw4ktrlpebwn1tom5urx6s
unzip -q data.zip
cd /content/
mv /content/tmp/MRF_Demo_ISMRM2022 /content/data
rm -rf tmp
mkdir /content/result
mkdir /content/result/synth/

In [ ]:
!git clone https://github.com/SophieSchau/MRF_demo_ISMRM2022.git
!pip install git+https://github.com/mikgroup/sigpy.git@master

In [ ]:
%%bash

cd /content/MRF_demo_ISMRM2022/src/01_calib/

python3 main.py --ksp /content/data/calib_raw.npy \
                --nse /content/data/calib_nse.npy \
                --nrc 40 --nsv 10 \
                --ccm /content/result/ccm.npy

In [ ]:
%%bash

cd /content/MRF_demo_ISMRM2022/src/02_recon/

python3 main.py -c --phi /content/data/subspace_basis_grp16.mat \
                   --ccm /content/result/ccm.npy \
                   --ksp /content/data/mrf_raw.npy \
                   --trj /content/data/traj_grp16_inacc2.mat \
                   --mtx 256 --ptt 10 --dev 0 --rnk 3 --mit 40 --nco 3 --akp False \
                   --res /content/result/fast.npy

In [ ]:
%%bash
cd /content/MRF_demo_ISMRM2022/src/03_synth/

python3 main.py  --inp /content/result/fast.npy \
                 --wgt /content/MRF_demo_ISMRM2022/src/03_synth/checkpoints/v1_InitCG/800_net_G.pth \
                 --sth /content/result/synth \
                 --idx 0 1 2 3 4

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

fast = np.load('/content/result/fast.npy')
synth = np.load('/content/result/synth/t1mprage.npy')
plt.imshow(np.abs(fast[:,:,128,0]))
plt.show()
plt.imshow(np.abs(synth[:,:,128]))